# Simulation for Salt intrusion


First the nessecary python packages are imported. This is a combination of open source packages (numpy, pygimli, ..) and code developed by Deltares (ert_inversion)

In [ ]:
import numpy as np
import subprocess

import pygimli as pg
from pygimli.meshtools import readGmsh
from pygimli.physics import ert

from ertoolbox import inversion
from ertoolbox import MPT

: 

## Mesh

To be able to simulate for the real geometry of the field measurements, the inversion mesh is created using GMSH and imported using Pygimli. The input file is .geo file and is rewritten into a .msh. After importing the mesh, the prior resistivity values are set in the K value per mesh domain. 

Creating a working mesh is very important, some key aspects that are vital for the inversion to work are:
- mesh should consists of points, lines, line loops and a plane surface, that are also made 'physical'
- line loop should be a closed loop
- electrode positions should be a point defined in the input file
- if you cannot make the mesh in gmsh already, the geometry is corrupt

We use 2 files. 1 file only has 1 fixed conductivity in the inversion domain. The other one has a fresh-salt interface and uses 2 different conductivities. 


The .geo file in imported, translated to .msh and filled with K values in this part of the code:

In [ ]:
filename_1l = "../data/Salt_intrusion_1_layer.geo"

geom_1l = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", "mesh.msh", filename_1l], shell=True)
mesh_1l = readGmsh("mesh.msh", verbose=True)

K_1l= [[2,100]]

pg.show(mesh_1l, markers=True, clipBoundaryMarkers=True)
pg.show(mesh_1l, data = K_1l, clipBoundaryMarkers=True, showmesh=True)

: 

In [ ]:
filename_2l = "../data/Salt_intrusion_2_layers.geo"
geom_2l = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", "mesh.msh", filename_2l], shell=True)
mesh_2l = readGmsh("mesh.msh", verbose=True)

K_2l= [[2,100],[3,15]]

pg.show(mesh_2l, markers=True, clipBoundaryMarkers=True)
pg.show(mesh_2l, data = K_2l, clipBoundaryMarkers=True, showmesh=True)

: 

for the inversion, we use a clean mesh.

In [ ]:
filename = "../data/Salt_intrusion_1_layer.geo"
geom = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", "mesh.msh", filename])
mesh = readGmsh("mesh.msh", verbose=True)

pg.show(mesh, markers=True, clipBoundaryMarkers=True)
pg.show(mesh,clipBoundaryMarkers=True, showmesh=True)

: 

## Simulate data

Using the custom mesh, we can make simulation data. This consists of the electrode positions and the inversion scheme. 3 schemes are shown here: dipole-dipole (dd), multiple gradient(gr) and one costom made mesh that was used in the field test.

Other possible standard configurations are: 
Wenner Alpha (C-P-P-C) :  wa
Wenner Beta(C-C-P-P) :  wb
Dipole Dipole (CC-PP) :  dd
Schlumberger(C-PP-C) :  slm
Pole Pole (C-P) :  pp
Pole Dipole (C-PP) :  pd
Half Wenner (C-P-P) :  hw
MultipleGradient(C--P-P--C) :  gr


first, the electrode positions are defined:

In [ ]:
EX = np.array([0.50,1.00,1.50,2.00,2.50,3.00,3.50,4.00,4.50,5.00,5.50,6.00,6.50,7.00,7.50,8.00,8.50,9.00,9.50,10.00,10.50,11.00,11.50,12.00,12.50,13.00,13.50,14.00,14.50,15.00,15.50,16.00,16.50,17.00,17.50,18.00,18.50,19.00,19.50,20.00,20.50,21.00,21.50,22.00,22.50,23.00,23.50,24.00,24.50,25.00,25.50,26.00,26.50,27.00,27.50,28.00,28.50,29.00,29.50,30.00,30.50,31.00,31.50,32.00,32.50,33.00,33.50,34.00,34.50,35.00,35.50,36.00,36.50,37.00,37.50,38.00,38.50,39.00,39.50,40.00,40.50,41.00,41.50,42.00,42.50,43.00,43.50,44.00,44.50,45.00,45.50,46.00,46.50,47.00])
EZ = np.array([-2.80,-2.98,-2.98,-2.98,-3.14,-3.14,-3.14,-3.40,-3.40,-3.54,-3.66,-3.66,-3.78,-3.90,-3.90,-4.03,-4.15,-4.25,-4.25,-4.33,-4.41,-4.41,-4.48,-4.55,-4.55,-4.69,-4.69,-4.69,-4.74,-4.74,-4.74,-4.68,-4.68,-4.62,-4.52,-4.52,-4.43,-4.33,-4.33,-4.28,-4.22,-4.19,-4.19,-4.14,-4.12,-4.12,-4.06,-4.07,-4.07,-4.09,-4.09,-4.09,-4.21,-4.21,-4.21,-4.27,-4.27,-4.40,-4.37,-4.37,-4.45,-4.43,-4.43,-4.53,-4.53,-4.62,-4.62,-4.62,-4.68,-4.68,-4.66,-4.68,-4.68,-4.59,-4.59,-4.59,-4.47,-4.47,-4.47,-4.33,-4.33,-4.28,-4.17,-4.17,-4.12,-4.00,-4.00,-3.94,-3.84,-3.84,-3.84,-3.84,-3.84,-3.84])

electrodes = [[x, y] for x, y in zip(EX, EZ)]

: 

### Dipole Dipole

The scheme is built-in and data is created using the dipole-dipole scheme for both geometries. Next, the simulated data is inverted using a clean inversion mesh.

In [ ]:
scheme_dd = ert.createData(elecs=electrodes, schemeName='dd')

simdata_dd_1l = ert.simulate(
         mesh=mesh_1l, scheme=scheme_dd, res=K_1l, noiseLevel=1, noiseAbs=1e-6, seed=1337
    )

simdata_dd_2l = ert.simulate(
         mesh=mesh_2l, scheme=scheme_dd, res=K_2l, noiseLevel=1, noiseAbs=1e-6, seed=1337
    )

simdata_dd_1l.remove(simdata_dd_1l['rhoa'] < 0)
simdata_dd_2l.remove(simdata_dd_2l['rhoa'] < 0)

inversion_dd_1 = inversion(simdata_dd_1l, mesh, saveresult=False)

inversion_dd_2 = inversion(simdata_dd_2l, mesh, saveresult=False)



: 

### Gradient
for the gradient based scheme, the workflow is the same as for dipole dipole.

In [ ]:
scheme_gr = ert.createData(elecs=electrodes, schemeName = 'gr')

simdata_gr_1l = ert.simulate(mesh=mesh_1l, scheme=scheme_gr, res=K_1l, noiseLevel=1, noiseAbs=1e-6, seed=1337)
simdata_gr_2l = ert.simulate(mesh=mesh_2l, scheme=scheme_gr, res=K_2l, noiseLevel=1, noiseAbs=1e-6, seed=1337)

simdata_gr_1l.remove(simdata_gr_1l['rhoa'] < 0)
simdata_gr_2l.remove(simdata_gr_2l['rhoa'] < 0)

inversion_gr_1 = inversion(simdata_gr_1l, mesh, saveresult=False)

inversion_gr_2 = inversion(simdata_gr_2l, mesh, saveresult=False)

: 

### Customized

For the customized scheme, this scheme should be created first and written down in a datacontainer for Pygimli to be able to handle the scheme.

In [ ]:
#datacontainer for custom abmn

import pygimli as pg
from pygimli.physics import ert
def prepare_custom_scheme(EX,EZ):
    dc = pg.DataContainerERT()

    for s in range(len(EX)):
        dc.createSensor(
            [
                EX[s],
                0,
                EZ[s],
            ]
        )
    
    A = [2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,78,78,78,78,78,78,78,78,78,78,78,78,78,78,79,79,79,79,79,79,79,79,79,79,79,79,79,80,80,80,80,80,80,80,80,80,80,80,80,80,81,81,81,81,81,81,81,81,81,81,81,81,81,82,82,82,82,82,82,82,82,82,82,82,82,83,83,83,83,83,83,83,83,83,83,83,84,84,84,84,84,84,84,84,84,84,85,85,85,85,85,85,85,85,85,86,86,86,86,86,86,86,86,87,87,87,87,87,87,87,88,88,88,88,88,88,88,89,89,89,89,89,89,90,90,90,90,90,91,91,91,91,92,92,92,93,93,94
]
    B = [1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,5,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,6,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7,7,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,8,8,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,8,9,9,9,9,9,9,9,7,7,7,7,7,7,8,8,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,10,10,8,8,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,10,11,11,11,11,11,11,11,9,9,9,9,9,9,10,10,10,10,10,10,11,11,11,11,11,11,12,12,12,12,12,12,12,10,10,10,10,10,10,11,11,11,11,11,11,12,12,12,12,12,12,13,13,13,13,13,13,13,11,11,11,11,11,11,12,12,12,12,12,12,13,13,13,13,13,13,14,14,14,14,14,14,14,12,12,12,12,12,12,13,13,13,13,13,13,14,14,14,14,14,14,15,15,15,15,15,15,15,13,13,13,13,13,13,14,14,14,14,14,14,15,15,15,15,15,15,16,16,16,16,16,16,16,14,14,14,14,14,14,15,15,15,15,15,15,16,16,16,16,16,16,17,17,17,17,17,17,17,15,15,15,15,15,15,16,16,16,16,16,16,17,17,17,17,17,17,18,18,18,18,18,18,18,16,16,16,16,16,16,17,17,17,17,17,17,18,18,18,18,18,18,19,19,19,19,19,19,19,17,17,17,17,17,17,18,18,18,18,18,18,19,19,19,19,19,19,20,20,20,20,20,20,20,18,18,18,18,18,18,19,19,19,19,19,19,20,20,20,20,20,20,21,21,21,21,21,21,21,19,19,19,19,19,19,20,20,20,20,20,20,21,21,21,21,21,21,22,22,22,22,22,22,22,20,20,20,20,20,20,21,21,21,21,21,21,22,22,22,22,22,22,23,23,23,23,23,23,23,21,21,21,21,21,21,22,22,22,22,22,22,23,23,23,23,23,23,24,24,24,24,24,24,24,22,22,22,22,22,22,23,23,23,23,23,23,24,24,24,24,24,24,25,25,25,25,25,25,25,23,23,23,23,23,23,24,24,24,24,24,24,25,25,25,25,25,25,26,26,26,26,26,26,26,24,24,24,24,24,24,25,25,25,25,25,25,26,26,26,26,26,26,27,27,27,27,27,27,27,25,25,25,25,25,25,26,26,26,26,26,26,27,27,27,27,27,27,28,28,28,28,28,28,28,26,26,26,26,26,26,27,27,27,27,27,27,28,28,28,28,28,28,29,29,29,29,29,29,29,27,27,27,27,27,27,28,28,28,28,28,28,29,29,29,29,29,29,30,30,30,30,30,30,30,28,28,28,28,28,28,29,29,29,29,29,29,30,30,30,30,30,30,31,31,31,31,31,31,31,29,29,29,29,29,29,30,30,30,30,30,30,31,31,31,31,31,31,32,32,32,32,32,32,32,30,30,30,30,30,30,31,31,31,31,31,31,32,32,32,32,32,32,33,33,33,33,33,33,33,31,31,31,31,31,31,32,32,32,32,32,32,33,33,33,33,33,33,34,34,34,34,34,34,34,32,32,32,32,32,32,33,33,33,33,33,33,34,34,34,34,34,34,35,35,35,35,35,35,35,33,33,33,33,33,33,34,34,34,34,34,34,35,35,35,35,35,35,36,36,36,36,36,36,36,34,34,34,34,34,34,35,35,35,35,35,35,36,36,36,36,36,36,37,37,37,37,37,37,37,35,35,35,35,35,35,36,36,36,36,36,36,37,37,37,37,37,37,38,38,38,38,38,38,38,36,36,36,36,36,36,37,37,37,37,37,37,38,38,38,38,38,38,39,39,39,39,39,39,39,37,37,37,37,37,37,38,38,38,38,38,38,39,39,39,39,39,39,40,40,40,40,40,40,40,38,38,38,38,38,38,39,39,39,39,39,39,40,40,40,40,40,40,41,41,41,41,41,41,41,39,39,39,39,39,39,40,40,40,40,40,40,41,41,41,41,41,41,42,42,42,42,42,42,42,40,40,40,40,40,40,41,41,41,41,41,41,42,42,42,42,42,42,43,43,43,43,43,43,43,41,41,41,41,41,41,42,42,42,42,42,42,43,43,43,43,43,43,44,44,44,44,44,44,44,42,42,42,42,42,42,43,43,43,43,43,43,44,44,44,44,44,44,45,45,45,45,45,45,45,43,43,43,43,43,43,44,44,44,44,44,44,45,45,45,45,45,45,46,46,46,46,46,46,46,44,44,44,44,44,44,45,45,45,45,45,45,46,46,46,46,46,46,47,47,47,47,47,47,47,45,45,45,45,45,45,46,46,46,46,46,46,47,47,47,47,47,47,48,48,48,48,48,48,48,46,46,46,46,46,46,47,47,47,47,47,47,48,48,48,48,48,48,49,49,49,49,49,49,49,47,47,47,47,47,47,48,48,48,48,48,48,49,49,49,49,49,49,50,50,50,50,50,50,50,48,48,48,48,48,48,49,49,49,49,49,49,50,50,50,50,50,50,51,51,51,51,51,51,51,49,49,49,49,49,49,50,50,50,50,50,50,51,51,51,51,51,51,52,52,52,52,52,52,52,50,50,50,50,50,50,51,51,51,51,51,51,52,52,52,52,52,52,53,53,53,53,53,53,53,51,51,51,51,51,51,52,52,52,52,52,52,53,53,53,53,53,53,54,54,54,54,54,54,54,52,52,52,52,52,52,53,53,53,53,53,53,54,54,54,54,54,54,55,55,55,55,55,55,55,53,53,53,53,53,53,54,54,54,54,54,54,55,55,55,55,55,55,56,56,56,56,56,56,56,54,54,54,54,54,54,55,55,55,55,55,55,56,56,56,56,56,56,57,57,57,57,57,57,57,55,55,55,55,55,55,56,56,56,56,56,56,57,57,57,57,57,57,58,58,58,58,58,58,58,56,56,56,56,56,56,57,57,57,57,57,57,58,58,58,58,58,58,59,59,59,59,59,59,59,57,57,57,57,57,57,58,58,58,58,58,58,59,59,59,59,59,59,60,60,60,60,60,60,60,58,58,58,58,58,58,59,59,59,59,59,59,60,60,60,60,60,60,61,61,61,61,61,61,61,59,59,59,59,59,59,60,60,60,60,60,60,61,61,61,61,61,61,62,62,62,62,62,62,62,60,60,60,60,60,60,61,61,61,61,61,61,62,62,62,62,62,62,63,63,63,63,63,63,63,61,61,61,61,61,61,62,62,62,62,62,62,63,63,63,63,63,63,64,64,64,64,64,64,64,62,62,62,62,62,63,63,63,63,63,63,64,64,64,64,64,64,65,65,65,65,65,65,65,63,63,63,63,64,64,64,64,64,64,65,65,65,65,65,65,66,66,66,66,66,66,66,64,64,64,65,65,65,65,65,65,66,66,66,66,66,66,67,67,67,67,67,67,67,65,65,66,66,66,66,66,66,67,67,67,67,67,67,68,68,68,68,68,68,68,66,67,67,67,67,67,67,68,68,68,68,68,68,69,69,69,69,69,69,69,68,68,68,68,68,68,69,69,69,69,69,69,70,70,70,70,70,70,70,69,69,69,69,69,69,70,70,70,70,70,70,71,71,71,71,71,71,71,70,70,70,70,70,70,71,71,71,71,71,71,72,72,72,72,72,72,72,71,71,71,71,71,72,72,72,72,72,72,73,73,73,73,73,73,73,72,72,72,72,73,73,73,73,73,73,74,74,74,74,74,74,74,73,73,73,74,74,74,74,74,74,75,75,75,75,75,75,75,74,74,75,75,75,75,75,75,76,76,76,76,76,76,76,75,76,76,76,76,76,76,77,77,77,77,77,77,77,77,77,77,77,77,77,78,78,78,78,78,78,78,78,78,78,78,78,78,79,79,79,79,79,79,79,79,79,79,79,79,79,80,80,80,80,80,80,80,80,80,80,80,80,81,81,81,81,81,81,81,81,81,81,81,82,82,82,82,82,82,82,82,82,82,83,83,83,83,83,83,83,83,83,84,84,84,84,84,84,84,84,85,85,85,85,85,85,85,86,86,86,86,86,86,86,87,87,87,87,87,87,87,88,88,88,88,88,88,89,89,89,89,89,90,90,90,90,91,91,91,92,92,93
]
    M = [3,4,5,6,7,8,9,11,12,13,14,15,16,4,5,6,7,8,9,10,19,20,21,22,23,24,12,13,14,15,16,17,5,6,7,8,9,10,11,27,28,29,30,31,32,20,21,22,23,24,25,13,14,15,16,17,18,6,7,8,9,10,11,12,28,29,30,31,32,33,21,22,23,24,25,26,14,15,16,17,18,19,7,8,9,10,11,12,13,29,30,31,32,33,34,22,23,24,25,26,27,15,16,17,18,19,20,8,9,10,11,12,13,14,30,31,32,33,34,35,23,24,25,26,27,28,16,17,18,19,20,21,9,10,11,12,13,14,15,31,32,33,34,35,36,24,25,26,27,28,29,17,18,19,20,21,22,10,11,12,13,14,15,16,32,33,34,35,36,37,25,26,27,28,29,30,18,19,20,21,22,23,11,12,13,14,15,16,17,33,34,35,36,37,38,26,27,28,29,30,31,19,20,21,22,23,24,1,1,1,1,1,1,1,34,35,36,37,38,39,27,28,29,30,31,32,2,2,2,2,2,2,1,1,1,1,1,1,1,35,36,37,38,39,40,2,2,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,1,2,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,2,2,3,3,3,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,1,1,1,1,2,2,2,3,3,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,1,1,1,2,2,2,2,3,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,3,1,1,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,3,3,1,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,4,2,2,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,5,3,3,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,3,4,4,4,4,5,5,3,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,3,3,4,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,3,3,3,4,4,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,3,3,3,3,4,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,4,2,2,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,4,4,2,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,5,3,3,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,5,5,3,4,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,6,4,4,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,4,5,5,5,5,6,6,4,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,4,4,5,5,5,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,4,4,4,5,5,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,4,4,4,4,5,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,5,3,3,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,5,5,3,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,6,4,4,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,6,6,4,5,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,7,5,5,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,5,6,6,6,6,7,7,5,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,5,5,6,6,6,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,5,5,5,6,6,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,5,5,5,5,6,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,6,4,4,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,6,6,4,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,7,5,5,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,7,7,5,6,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,8,6,6,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,6,7,7,7,7,8,8,6,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,6,6,7,7,7,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,6,6,6,7,7,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,6,6,6,6,7,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,7,5,5,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,7,7,5,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,8,6,6,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,8,8,6,7,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,9,7,7,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,7,8,8,8,8,9,9,7,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,7,7,7,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,7,7,7,7,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,8,6,6,7,7,7,7,7,9,9,9,8,8,8,8,8,8,7,7,7,7,8,8,6,7,7,7,7,7,7,9,9,8,8,8,8,8,8,7,7,7,8,8,8,7,7,7,7,7,7,7,9,8,8,8,8,8,9,7,7,8,8,8,8,7,7,7,7,7,7,7,8,8,8,8,9,9,7,8,8,8,8,8,7,7,7,7,7,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,8,8,8,8,9,9,8,8,8,8,8,9,7,7,8,8,8,8,8,9,8,8,8,8,9,9,7,8,8,8,8,8,8,8,8,8,9,9,9,8,8,8,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,8,8,8,8,8,8,8,9,9,9,9,9,8,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,9,9,9,9,8,8,8,8,9,9,9,9,8,8,8,9,9,9,9,8,8,9,9,9,9,9,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
]
    N = [4,5,6,7,8,9,10,13,14,15,16,17,18,5,6,7,8,9,10,11,22,23,24,25,26,27,14,15,16,17,18,19,6,7,8,9,10,11,12,30,31,32,33,34,35,23,24,25,26,27,28,15,16,17,18,19,20,7,8,9,10,11,12,13,31,32,33,34,35,36,24,25,26,27,28,29,16,17,18,19,20,21,8,9,10,11,12,13,14,32,33,34,35,36,37,25,26,27,28,29,30,17,18,19,20,21,22,9,10,11,12,13,14,15,33,34,35,36,37,38,26,27,28,29,30,31,18,19,20,21,22,23,10,11,12,13,14,15,16,34,35,36,37,38,39,27,28,29,30,31,32,19,20,21,22,23,24,11,12,13,14,15,16,17,3,3,3,3,3,4,2,2,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,1,1,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,2,3,4,5,6,7,8,3,3,3,4,4,4,3,3,3,3,3,3,0,1,2,3,4,5,3,4,5,6,7,8,9,3,3,4,4,4,4,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,9,0,1,2,3,2,3,4,5,6,7,5,6,7,8,9,0,1,0,1,2,3,3,4,5,6,7,8,6,7,8,9,0,1,2,1,2,3,4,5,6,7,8,9,7,8,9,0,1,2,3,2,3,5,6,7,8,9,0,8,9,0,1,2,3,4,3,6,7,8,9,0,1,9,0,1,2,3,4,5,7,8,9,0,1,2,0,1,2,3,4,5,6,8,9,0,1,2,3,1,2,3,4,5,6,7,9,0,1,2,3,4,2,3,4,5,6,7,8,0,1,2,3,4,3,4,5,6,7,8,9,1,2,3,4,4,5,6,7,8,9,0,2,3,4,5,6,7,8,9,0,1,3,4,6,7,8,9,0,1,2,4,7,8,9,0,1,2,3,8,9,0,1,2,3,4,9,0,1,2,3,4,5,0,1,2,3,4,5,1,2,3,4,5,2,3,4,5,3,4,5,4,5,5
]
    
    for i in range(len(A)):
        dc.addFourPointData(
            A[i]-1,
            B[i]-1,
            M[i]-1,
            N[i]-1,
        )
    k = ert.createGeometricFactors(dc)
    dc.set("k", k)

    return dc

dc = prepare_custom_scheme(EX,EZ)


: 

then, a similar approach of making the data and inverting the data is carried out.

In [ ]:
simdata_c_1l = ert.simulate(mesh=mesh_1l, scheme=dc, res=K_1l, noiseLevel=1, noiseAbs=1e-6, seed=1337)
simdata_c_2l = ert.simulate(mesh=mesh_2l, scheme=dc, res=K_2l, noiseLevel=1, noiseAbs=1e-6, seed=1337)

simdata_c_1l.remove(simdata_c_1l['rhoa'] < 0)
simdata_c_2l.remove(simdata_c_2l['rhoa'] < 0)

inversion_c_1 = inversion(simdata_c_1l, mesh, saveresult=False)
inversion_c_2 = inversion(simdata_c_2l, mesh, saveresult=False)

: 

: 